## 라이브러리 import

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import tensorflow as tf
print(tf.__version__)

Tensorflow v1 기능만 사용 (v2 기능 강제 차단)

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(777)

## 데이터 불러오기

In [ ]:
Fold = 5

# k-fold 학습/검증 데이터
for i in range(Fold):
    
    path1 = './K_FoldData/Training_Fold%d'%(i+1)
    path2 = './K_FoldData/Validation_Fold%d'%(i+1)
    c1 = 'Training_Fold%d   = np.array(pd.read_csv(path1, sep=",", header=None))'%(i+1)
    c2 = 'Validation_Fold%d = np.array(pd.read_csv(path2, sep=",", header=None))'%(i+1)
    exec(c1)
    exec(c2)

# K-fold 학습/검증 레이블
TrainingFold_Label   = np.array(pd.read_csv('./K_FoldData/TrainingFold_Label_forANN'  , sep=",", header=None))
ValidationFold_Label = np.array(pd.read_csv('./K_FoldData/ValidationFold_Label_forANN', sep=",", header=None))
    
    
# 전체 학습용 데이터
Training_All       = np.array(pd.read_csv('./K_FoldData/Training_All', sep = ",", header = None))
Training_All_Label = np.array(pd.read_csv('./K_FoldData/Training_All_Label_forANN', sep = ",", header = None))

print(Training_Fold1.shape)
print(Validation_Fold1.shape)
print(TrainingFold_Label.shape)
print(ValidationFold_Label.shape)
print(Training_All.shape)

## ANN(Artificial Neural Network) hyperparameter 설정

In [ ]:
learningRate  = 0.0001
noOfNeuron    = 20
iteration     = 5000

## ANN의 구조(Architecture) 설계

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## NN(Neural Network) 학습 및 검증

In [ ]:
Label = TrainingFold_Label
Label_Val = ValidationFold_Label

for i in range(Fold):
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

# train my model
# initialize
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    for step in range(1, iteration+1):
    
        c, _ = sess.run([cost, optimizer], feed_dict = {X : Data, Y: Label})
        s2 = 'Validation_Acc_Fold%d = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})'%(i+1)
        exec(s2)

Accuracy_sum = 0

for j in range(Fold):
    
    s3 = 'Accuracy_sum += Validation_Acc_Fold%d'%(j+1)
    exec(s3)
    
print('[Reslut of K-fold Cross Validation] \n')
print('Fold 1: {:.5f} \nFold 2: {:.5f} \nFold 3: {:.5f} \nFold 4: {:.5f} \nFold 5: {:.5f} \n\n * Average accuracy : {:.5f}'
      .format(Validation_Acc_Fold1, Validation_Acc_Fold2, Validation_Acc_Fold3, Validation_Acc_Fold4, Validation_Acc_Fold5, Accuracy_sum/Fold))

## 전체 학습 데이터로 NN 학습 및 모델 저장

In [ ]:
x_data = Training_All
y_data = Training_All_Label

In [ ]:
# train my model
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(1, iteration+1):
    
    c, _ = sess.run([cost, optimizer], feed_dict = {X : x_data, Y: y_data})
    Acc = sess.run(accuracy, feed_dict={X : x_data, Y: y_data})
   
    if step % 1000 == 0:
            
        print('iteration {}'.format(step))    
        print('   Validation Acc: {:.5f}'.format(Acc))

In [ ]:
saver = tf.train.Saver()
saver.save(sess, './MLmodels/ANN_model_tf1')

## NN model 불러와서 진단

텐서플로우 그래프 초기화 (아무것도 작성안했을 시 실행 X)

In [ ]:
# Tensorflow Graph 지울 때 사용
tf.reset_default_graph()

In [ ]:
# input place holders
X = tf.placeholder(tf.float32, [None, Training_Fold1.shape[1]])
Y = tf.placeholder(tf.float32, [None, 2])

# dropout (keep_prob) rate 0.7 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# weights & bias for nn layers

# Input Layer
W1 = tf.Variable(tf.random_normal([Training_Fold1.shape[1], noOfNeuron]))
b1 = tf.Variable(tf.random_normal([noOfNeuron]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# Hidden Layer 1
W2 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b2 = tf.Variable(tf.random_normal([noOfNeuron]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

# Hidden Layer 2
W3 = tf.Variable(tf.random_normal([noOfNeuron, noOfNeuron]))
b3 = tf.Variable(tf.random_normal([noOfNeuron]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

# Output Layer
W4 = tf.Variable(tf.random_normal([noOfNeuron, 2]))
b4 = tf.Variable(tf.random_normal([2]))
hypothesis = tf.matmul(L3, W4) + b4


# define cost/loss &optimizer
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy           = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
# 모델 불러오기 (그래프에 Variable w, b 불러옴)
sess  = tf.Session()
LoadedModel = tf.train.Saver()
LoadedModel.restore(sess, './MLmodels/ANN_model_tf1')

In [ ]:
for i in range(Fold):
    
    s1= 'Data = Training_Fold%d'%(i+1)
    s2= 'Data_Val = Validation_Fold%d'%(i+1)
    exec(s1)
    exec(s2)

    s3 = 'Validation_Acc_Fold%d = sess.run(accuracy, feed_dict={X : Data_Val, Y: Label_Val})'%(i+1)
    exec(s3)

Accuracy_sum = 0

for j in range(Fold):
    
    s3 = 'Accuracy_sum += Validation_Acc_Fold%d'%(j+1)
    exec(s3)

print('[Performance of ANN(tf1) model] \n')
print('Fold 1: {:.5f} \nFold 2: {:.5f} \nFold 3: {:.5f} \nFold 4: {:.5f} \nFold 5: {:.5f} \n\n * Average accuracy : {:.5f}'
      .format(Validation_Acc_Fold1, Validation_Acc_Fold2, Validation_Acc_Fold3, Validation_Acc_Fold4, Validation_Acc_Fold5, Accuracy_sum/Fold))